In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [2]:
from diffusion.data_loaders.spinkick_diffuser_dataset import SpinkickDiffuserDataset
dataset = SpinkickDiffuserDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_spinkick.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8251,  ..., -0.5877, -0.5578, -0.1123],
         [-0.0098, -0.0053,  0.8223,  ..., -0.6034, -0.5496, -0.1377],
         [-0.0205, -0.0135,  0.8187,  ..., -0.6073, -0.5372, -0.1672],
         ...,
         [ 0.2570, -0.0458,  0.8492,  ..., -0.3039, -0.5771, -0.0356],
         [ 0.2659, -0.0516,  0.8421,  ..., -0.2956, -0.5725, -0.0793],
         [ 0.2787, -0.0612,  0.8371,  ..., -0.3081, -0.5633, -0.1231]]), conditions={0: tensor([ 0.0000,  0.0000,  0.8251, -0.9969,  0.0457, -0.0639, -0.0029, -0.0399,
          0.0621, -0.0921,  0.0930,  0.4230,  0.1155, -1.4425,  0.4709, -0.4550,
          1.1228,  1.3071, -0.7625, -0.2475,  1.4452, -0.1059, -0.4803,  0.0466,
         -0.8114,  0.0888, -0.3455, -0.1734,  0.2040, -0.1554,  0.3841, -0.6268,
         -0.5877, -0.5578, -0.1123])}),
 torch.Size([64, 35]))

### Model NN

In [14]:
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

savepath = '/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [15]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
horizon=64
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 64
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [16]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

n_timesteps = 20
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 64
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 20
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2/diffusion_config.pkl



In [17]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2/trainer_config.pkl



In [18]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 64, 35])
x.shape torch.Size([1, 35, 64])
x.shape torch.Size([1, 32, 64])
1
xfinal.shape torch.Size([1, 32, 32])
x.shape torch.Size([1, 64, 32])
2
xfinal.shape torch.Size([1, 64, 16])
x.shape torch.Size([1, 128, 16])
3
xfinal.shape torch.Size([1, 128, 8])
x.shape torch.Size([1, 256, 8])
4
xfinal.shape torch.Size([1, 256, 8])
xt1.shape torch.Size([1, 256, 8])
xt2.shape torch.Size([1, 256, 8])
xt3.shape torch.Size([1, 256, 8])
pop.shape torch.Size([1, 256, 8])
pop.shape torch.Size([1, 128, 16])
pop.shape torch.Size([1, 64, 32])


(tensor([[[-0.2419,  0.2646,  0.0311,  ...,  0.0439, -0.2698, -0.0281],
          [-0.1040,  0.1145,  0.1460,  ..., -0.2028, -0.2967,  0.2393],
          [-0.0521,  0.1151, -0.0678,  ...,  0.0525, -0.0746,  0.1755],
          ...,
          [-0.1967,  0.2516,  0.0063,  ..., -0.5132,  0.1369,  0.0428],
          [ 0.4825,  0.5278,  0.0043,  ...,  0.1212, -0.2280,  0.1019],
          [-0.0719,  0.2038,  0.2535,  ...,  0.3998,  0.4423, -0.2622]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 64, 35]))

In [19]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

In [20]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2/state_0.pt
0:   0.4112 | a0_loss:   0.1079 | t:   0.4738
100:   0.0828 | a0_loss:   0.0111 | t:   8.3614
200:   0.0371 | a0_loss:   0.0046 | t:   8.6597
300:   0.0192 | a0_loss:   0.0029 | t:   8.4928
400:   0.0108 | a0_loss:   0.0020 | t:   8.7196
500:   0.0066 | a0_loss:   0.0014 | t:   8.8373
600:   0.0043 | a0_loss:   0.0011 | t:   8.8342
700:   0.0030 | a0_loss:   0.0008 | t:   8.7446
800:   0.0021 | a0_loss:   0.0006 | t:   8.2157
900:   0.0016 | a0_loss:   0.0005 | t:   8.2309
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser2/state_1.pt


In [21]:
# from diffusion.diffuser.utils import load_diffusion

# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

trainer.load(1)
model = trainer.ema_model

In [22]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


                                                                                                    
1 / 20 [###                                                         ]   5% | 3.6 Hz
t : 19 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 20 [######                                                      ]  10% | 7.0 Hz
t : 18 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 20 [#########                                                   ]  15% | 10.2 Hz
t : 17 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                         

In [23]:
sample, sample.trajectories.shape

(Sample(trajectories=tensor([[[ 2.5701e-02,  1.1854e-02,  8.7868e-01,  ..., -5.8522e-01,
           -5.4016e-01, -9.9224e-02],
          [ 8.1862e-04, -5.3689e-04,  8.8429e-01,  ..., -5.8408e-01,
           -5.3077e-01, -1.2169e-01],
          [ 1.8072e-02, -1.2620e-02,  8.2416e-01,  ..., -5.5923e-01,
           -5.4517e-01, -1.2093e-01],
          ...,
          [ 2.8107e-01, -3.1115e-02,  7.7258e-01,  ..., -3.2987e-01,
           -5.2195e-01, -9.0142e-03],
          [ 2.7092e-01, -3.1723e-02,  8.2494e-01,  ..., -3.3930e-01,
           -5.2596e-01, -3.5684e-02],
          [ 2.8167e-01, -4.0632e-03,  8.1294e-01,  ..., -3.3599e-01,
           -5.3855e-01, -4.1234e-02]]], device='cuda:0'), values=tensor([0.], device='cuda:0'), chains=None),
 torch.Size([1, 64, 35]))

In [24]:
dataset[0], dataset[0].trajectories.shape

(Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8251,  ..., -0.5877, -0.5578, -0.1123],
         [-0.0098, -0.0053,  0.8223,  ..., -0.6034, -0.5496, -0.1377],
         [-0.0205, -0.0135,  0.8187,  ..., -0.6073, -0.5372, -0.1672],
         ...,
         [ 0.2570, -0.0458,  0.8492,  ..., -0.3039, -0.5771, -0.0356],
         [ 0.2659, -0.0516,  0.8421,  ..., -0.2956, -0.5725, -0.0793],
         [ 0.2787, -0.0612,  0.8371,  ..., -0.3081, -0.5633, -0.1231]]), conditions={0: tensor([ 0.0000,  0.0000,  0.8251, -0.9969,  0.0457, -0.0639, -0.0029, -0.0399,
          0.0621, -0.0921,  0.0930,  0.4230,  0.1155, -1.4425,  0.4709, -0.4550,
          1.1228,  1.3071, -0.7625, -0.2475,  1.4452, -0.1059, -0.4803,  0.0466,
         -0.8114,  0.0888, -0.3455, -0.1734,  0.2040, -0.1554,  0.3841, -0.6268,
         -0.5877, -0.5578, -0.1123])}),
 torch.Size([64, 35]))

In [26]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion {i} saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="motion1.npy")

Motion 0 saved as motion1.npy
